# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-10724000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Bidding

In [2]:
vuln_ns, vuln_ew = False, True
hand = 'A.8xx.Jxx.AQxxxx'

auction = ["1S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?


[{'call': 'PASS',
  'insta_score': 0.958,
  'expected_score': -123,
  'adjustment': 48},
 {'call': '2C', 'insta_score': 0.038, 'expected_score': -138, 'adjustment': 2}]

In [3]:
auction = ["1S","PASS","1N","PASS","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']


[{'call': 'X', 'insta_score': 0.943, 'expected_score': 71, 'adjustment': 47},
 {'call': 'PASS', 'insta_score': 0.047, 'expected_score': 21, 'adjustment': 2}]

In [4]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


KQJxx.Q.Axx.K9xx A.8xx.Jxx.AQxxxx 8x.J9xx.KQ8x.JT8 T9xxx.AKTxx.T9x. 0.99947 
 1S-P-1N-P-2C-X-P-P-P (6.2)  
 1S-P-1N-P-2C-P-2S-P-P-P (6.6) 
Qxxxx.ATx.A.KJT8 A.8xx.Jxx.AQxxxx K8.KJ9x.QT98.9xx JT9xx.Qxx.Kxxxx. 0.99937 
 1S-P-1N-P-2C-X-2S-P-P-P (7.6)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.6) 
KJ8xx.QTx.A.KJxx A.8xx.Jxx.AQxxxx Tx.Axx.KT98x.T98 Q9xxx.KJ9x.Qxxx. 0.99919 
 1S-P-1N-P-2C-X-P-2H-P-P-P (6.7)  
 1S-P-1N-P-2C-P-2S-P-P-P (6.9) 
KJxxx.A.KTx.KJ9x A.8xx.Jxx.AQxxxx xx.QJTxx.A9xx.8x QT98x.K9xx.Q8x.T 0.99895 
 1S-P-1N-P-2C-X-P-P-P (6.3)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.5) 
K9xxx.x.KQx.KJ8x A.8xx.Jxx.AQxxxx 8x.QTxxx.AT8x.T9 QJTxx.AKJ9.9xx.x 0.99874 
 1S-P-1N-P-2C-X-P-2H-P-P-P (8.1)  
 1S-P-1N-P-2C-P-P-X-P-3C-P-3H-P-P-P (8.1) 
QJ9xx.A.ATx.KJT8 A.8xx.Jxx.AQxxxx Kx.KQJxx.9xxx.xx T8xxx.T9xx.KQ8.9 0.99873 
 1S-P-1N-P-2C-X-P-P-P (6.2)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.7) 
QT9xx.Axx.K.KJxx A.8xx.Jxx.AQxxxx Kx.QT9x.QT8xx.T9 J8xxx.KJx.A9xx.8 0.99854 
 1S-P-1N-P-2C-X-2S-P-P-P (6.3)  
 1S-P-1N-P-2C-P-2S-P-P-P (6.3) 


West always has the same cards because it's your hand and it's known to you

In [5]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(auction)

for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

T8xxx.KJx.KT9x.9 KQ.ATx.AQ8xx.Jxx A.8xx.Jxx.AQxxxx J9xxx.Q9xx.x.KT8 1.00
9xx.xxxx.Qx.KT8x Jxx.AKT9.AK8x.J9 A.8xx.Jxx.AQxxxx KQT8xx.QJ.T9xx.x 1.00
T9xx.QJ9.K9x.Txx KJ8.Kxxx.AQxx.K9 A.8xx.Jxx.AQxxxx Qxxxx.ATx.T8x.J8 1.00
Qxx.T9xx.KTx.J9x KT8x.AKx.AQxx.T8 A.8xx.Jxx.AQxxxx J9xxx.QJx.98x.Kx 1.00
T9xx.KJ.T9xx.T8x Qxx.AT9x.AK8x.Kx A.8xx.Jxx.AQxxxx KJ8xx.Qxxx.Qx.J9 1.00
KT9xxx.Axxxx.xx. Qxx.KQ.A98xx.KJ8 A.8xx.Jxx.AQxxxx J8x.JT9.KQT.T9xx 0.98
Qxxx.Axx.K9xx.J9 KJT9x.QT.AQT.K8x A.8xx.Jxx.AQxxxx 8xx.KJ9xx.8xx.Tx 0.35
Kxxx.AJxx.Q9x.T8 JT8x.KQTx.AKx.KJ A.8xx.Jxx.AQxxxx Q9xx.9x.T8xx.9xx 1.00
JTx.Txxx.Qxx.T9x Q9x.K9.AK9x.KJ8x A.8xx.Jxx.AQxxxx K8xxxx.AQJx.T8x. 1.00
xxxx.Q9xx.9xx.Kx QJxx.AKx.AQxx.Tx A.8xx.Jxx.AQxxxx KT98.JTx.KT8.J98 1.00
